In [13]:
from ete3 import Tree, TreeStyle, TextFace
import PyQt5
import rst_lib
from rst_lib import TRAIN, DOUBLE
from ete3 import NodeStyle

In [26]:
ts = TreeStyle()
ts.show_leaf_name = False
ns = NodeStyle()
ns['size'] = 0


def get_newick_helper(subtree):
  if subtree.is_leaf:
    return str(subtree.tags['span_leaf'])
  else:
    return "(" + ", ".join([get_newick_helper(child) for child in subtree.children_in_order()]) +")"


def get_newick(subtree):
  return get_newick_helper(subtree) + ";"
  
paths, files = rst_lib.build_file_map()

annotation_pairs = [
    rst_lib.build_annotation_pair(files, paths, identifier)
    for identifier in files[TRAIN][DOUBLE]
]

for p in annotation_pairs:
  if p.main_annotation.is_valid and p.double_annotation.is_valid:
    maps = {
      "main": p.main_annotation,
      "double": p.double_annotation
    }
    for which_annotation, annotation in maps.items():
      newick = get_newick(annotation.tree)
      face_map = [None] + [" ".join(edu) for edu in annotation.edus[1:]]
      t = Tree(newick)
      for x in t.traverse():
          x.set_style(ns)
          if x.name:
              x.add_face(TextFace("  " +face_map[int(x.name)]), 0)
      t.convert_to_ultrametric()
      t.render(f"{p.identifier}_{which_annotation}.pdf", tree_style=ts, w=700, dpi=300)